# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f31a49a0d90>
├── 'a' --> tensor([[-2.7781, -0.3651,  0.3574],
│                   [-0.1617, -0.9157, -0.8280]])
└── 'x' --> <FastTreeValue 0x7f31a4993310>
    └── 'c' --> tensor([[ 1.5213, -0.0544,  0.1993, -0.2286],
                        [ 0.8726, -0.3535, -0.6794, -0.0687],
                        [-1.2833, -0.4827,  0.7666, -1.1159]])

In [4]:
t.a

tensor([[-2.7781, -0.3651,  0.3574],
        [-0.1617, -0.9157, -0.8280]])

In [5]:
%timeit t.a

66.2 ns ± 0.0267 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f31a49a0d90>
├── 'a' --> tensor([[-0.2031,  2.1038, -0.0236],
│                   [-0.0651, -0.3515,  1.9468]])
└── 'x' --> <FastTreeValue 0x7f31a4993310>
    └── 'c' --> tensor([[ 1.5213, -0.0544,  0.1993, -0.2286],
                        [ 0.8726, -0.3535, -0.6794, -0.0687],
                        [-1.2833, -0.4827,  0.7666, -1.1159]])

In [7]:
%timeit t.a = new_value

62.6 ns ± 0.0601 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-2.7781, -0.3651,  0.3574],
               [-0.1617, -0.9157, -0.8280]]),
    x: Batch(
           c: tensor([[ 1.5213, -0.0544,  0.1993, -0.2286],
                      [ 0.8726, -0.3535, -0.6794, -0.0687],
                      [-1.2833, -0.4827,  0.7666, -1.1159]]),
       ),
)

In [10]:
b.a

tensor([[-2.7781, -0.3651,  0.3574],
        [-0.1617, -0.9157, -0.8280]])

In [11]:
%timeit b.a

55.9 ns ± 0.0129 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 1.3712,  0.7645, -1.7939],
               [-0.8566,  0.3959,  0.5263]]),
    x: Batch(
           c: tensor([[ 1.5213, -0.0544,  0.1993, -0.2286],
                      [ 0.8726, -0.3535, -0.6794, -0.0687],
                      [-1.2833, -0.4827,  0.7666, -1.1159]]),
       ),
)

In [13]:
%timeit b.a = new_value

482 ns ± 0.383 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

867 ns ± 35 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.3 µs ± 51.7 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

137 µs ± 1.07 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

140 µs ± 472 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f30eec48f10>
├── 'a' --> tensor([[[-2.7781, -0.3651,  0.3574],
│                    [-0.1617, -0.9157, -0.8280]],
│           
│                   [[-2.7781, -0.3651,  0.3574],
│                    [-0.1617, -0.9157, -0.8280]],
│           
│                   [[-2.7781, -0.3651,  0.3574],
│                    [-0.1617, -0.9157, -0.8280]],
│           
│                   [[-2.7781, -0.3651,  0.3574],
│                    [-0.1617, -0.9157, -0.8280]],
│           
│                   [[-2.7781, -0.3651,  0.3574],
│                    [-0.1617, -0.9157, -0.8280]],
│           
│                   [[-2.7781, -0.3651,  0.3574],
│                    [-0.1617, -0.9157, -0.8280]],
│           
│                   [[-2.7781, -0.3651,  0.3574],
│                    [-0.1617, -0.9157, -0.8280]],
│           
│                   [[-2.7781, -0.3651,  0.3574],
│                    [-0.1617, -0.9157, -0.8280]]])
└── 'x' --> <FastTreeValue 0x7f30eec48be0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.6 µs ± 56.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f30f5c68b20>
├── 'a' --> tensor([[-2.7781, -0.3651,  0.3574],
│                   [-0.1617, -0.9157, -0.8280],
│                   [-2.7781, -0.3651,  0.3574],
│                   [-0.1617, -0.9157, -0.8280],
│                   [-2.7781, -0.3651,  0.3574],
│                   [-0.1617, -0.9157, -0.8280],
│                   [-2.7781, -0.3651,  0.3574],
│                   [-0.1617, -0.9157, -0.8280],
│                   [-2.7781, -0.3651,  0.3574],
│                   [-0.1617, -0.9157, -0.8280],
│                   [-2.7781, -0.3651,  0.3574],
│                   [-0.1617, -0.9157, -0.8280],
│                   [-2.7781, -0.3651,  0.3574],
│                   [-0.1617, -0.9157, -0.8280],
│                   [-2.7781, -0.3651,  0.3574],
│                   [-0.1617, -0.9157, -0.8280]])
└── 'x' --> <FastTreeValue 0x7f31a49932e0>
    └── 'c' --> tensor([[ 1.5213, -0.0544,  0.1993, -0.2286],
                        [ 0.8726, -0.3535, -0.6794, -0.0687],
                 

In [23]:
%timeit t_cat(trees)

30.4 µs ± 37.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

58.1 µs ± 153 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-2.7781, -0.3651,  0.3574],
                [-0.1617, -0.9157, -0.8280]],
       
               [[-2.7781, -0.3651,  0.3574],
                [-0.1617, -0.9157, -0.8280]],
       
               [[-2.7781, -0.3651,  0.3574],
                [-0.1617, -0.9157, -0.8280]],
       
               [[-2.7781, -0.3651,  0.3574],
                [-0.1617, -0.9157, -0.8280]],
       
               [[-2.7781, -0.3651,  0.3574],
                [-0.1617, -0.9157, -0.8280]],
       
               [[-2.7781, -0.3651,  0.3574],
                [-0.1617, -0.9157, -0.8280]],
       
               [[-2.7781, -0.3651,  0.3574],
                [-0.1617, -0.9157, -0.8280]],
       
               [[-2.7781, -0.3651,  0.3574],
                [-0.1617, -0.9157, -0.8280]]]),
    x: Batch(
           c: tensor([[[ 1.5213, -0.0544,  0.1993, -0.2286],
                       [ 0.8726, -0.3535, -0.6794, -0.0687],
                       [-1.2833, -0.4827,  0.7666, -1.1159]],
         

In [26]:
%timeit Batch.stack(batches)

77.9 µs ± 261 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-2.7781, -0.3651,  0.3574],
               [-0.1617, -0.9157, -0.8280],
               [-2.7781, -0.3651,  0.3574],
               [-0.1617, -0.9157, -0.8280],
               [-2.7781, -0.3651,  0.3574],
               [-0.1617, -0.9157, -0.8280],
               [-2.7781, -0.3651,  0.3574],
               [-0.1617, -0.9157, -0.8280],
               [-2.7781, -0.3651,  0.3574],
               [-0.1617, -0.9157, -0.8280],
               [-2.7781, -0.3651,  0.3574],
               [-0.1617, -0.9157, -0.8280],
               [-2.7781, -0.3651,  0.3574],
               [-0.1617, -0.9157, -0.8280],
               [-2.7781, -0.3651,  0.3574],
               [-0.1617, -0.9157, -0.8280]]),
    x: Batch(
           c: tensor([[ 1.5213, -0.0544,  0.1993, -0.2286],
                      [ 0.8726, -0.3535, -0.6794, -0.0687],
                      [-1.2833, -0.4827,  0.7666, -1.1159],
                      [ 1.5213, -0.0544,  0.1993, -0.2286],
                      [ 0.8726, -

In [28]:
%timeit Batch.cat(batches)

142 µs ± 110 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

306 µs ± 981 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
